In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5103)
Using Theano backend.


In [2]:
#path = "data/ml-20m/"
path = "data/ml-small/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

In [8]:
ratings = pd.read_csv(path+'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [10]:
ratings.shape

(100004, 4)

In [154]:
# movies = pd.read_csv(path+'movies.csv')
# movies.head()
movie_names = pd.read_csv(path+'movies.csv').set_index('movieId')['title'].to_dict()
# movie_names

In [155]:
userIds = ratings.userId.unique()
movieIds = ratings.movieId.unique()

In [24]:
userIdToIdx = {uId: idx for idx, uId in enumerate(userIds)}
movieIdToIdx = {mId: idx for idx, mId in enumerate(movieIds)}

In [26]:
ratings['movieIdx'] = ratings.movieId.apply(lambda x: movieIdToIdx[x])
ratings['userIdx'] = ratings.userId.apply(lambda x: userIdToIdx[x])

In [28]:
ratings.head()

,userId,movieId,rating,timestamp,movieIdx,userIdx
0,1,31,2.5,1260759144,0,0
1,1,1029,3.0,1260759179,1,0
2,1,1061,3.0,1260759182,2,0
3,1,1129,2.0,1260759185,3,0
4,1,1172,4.0,1260759205,4,0


In [145]:
user_min, user_max, movie_min, movie_max = (ratings.userIdx.min(), ratings.userIdx.max(), 
                                            ratings.movieIdx.min(), ratings.movieIdx.max())
user_min, user_max, movie_min, movie_max

(0, 670, 0, 9065)

In [38]:
n_users = ratings.userIdx.nunique()
n_movies = ratings.movieIdx.nunique()
n_users, n_movies

(671, 9066)

In [39]:
n_factors = 50
np.random.seed = 42

In [42]:
msk = np.random.rand(len(ratings)) < 0.8
trn = ratings[msk]
val = ratings[~msk]

In [44]:
trn.head()

,userId,movieId,rating,timestamp,movieIdx,userIdx
2,1,1061,3.0,1260759182,2,0
4,1,1172,4.0,1260759205,4,0
7,1,1293,2.0,1260759148,7,0
8,1,1339,3.5,1260759125,8,0
9,1,1343,2.0,1260759131,9,0


In [45]:
val.head()

,userId,movieId,rating,timestamp,movieIdx,userIdx
0,1,31,2.5,1260759144,0,0
1,1,1029,3.0,1260759179,1,0
3,1,1129,2.0,1260759185,3,0
5,1,1263,2.0,1260759151,5,0
6,1,1287,2.0,1260759187,6,0


## Excel

In [63]:
g = ratings.groupby('userIdx')['rating'].count()
topUsers = g.sort_values(ascending=False)[:5]

In [64]:
g = ratings.groupby('movieIdx')['rating'].count()
topMovies = g.sort_values(ascending=False)[:5]
topMovies.head()

movieIdx
57     341
49     324
99     311
92     304
143    291
Name: rating, dtype: int64

In [65]:
top_r = ratings.join(topUsers, rsuffix='_r', how='inner', on='userIdx')

In [66]:
top_r = top_r.join(topMovies, rsuffix='_r', how='inner', on='movieIdx')
top_r

,userId,movieId,rating,timestamp,movieIdx,userIdx,rating_r,rating_r
1032,15,260,5.0,997938437,143,14,1700,291
10275,73,260,4.5,1255508202,143,72,1610,291
82976,564,260,2.0,974710704,143,563,1868,291
93571,624,260,5.0,1019124071,143,623,1735,291
1037,15,296,5.0,997938771,49,14,1700,324
10281,73,296,5.0,1255502790,49,72,1610,324
78490,547,296,5.0,981314122,49,546,2391,324
82991,564,296,5.0,974711078,49,563,1868,324
93573,624,296,5.0,1019132316,49,623,1735,324
1044,15,318,2.0,997938727,99,14,1700,311


In [146]:
pd.crosstab(top_r.userIdx, top_r.movieIdx, top_r.rating, aggfunc=np.mean)

movieIdx,49,57,92,99,143
userIdx,,,,,
14,5.0,1.0,5.0,2.0,5.0
72,5.0,5.0,4.5,5.0,4.5
546,5.0,2.0,5.0,5.0,NaN
563,5.0,3.0,5.0,NaN,2.0
623,5.0,3.0,5.0,NaN,5.0


## Dot product

In [78]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [89]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
model = Model([user_in, movie_in], x)
model.compile(Adam(1e-3), loss='mse', metrics=['accuracy'])


In [90]:
model.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=1,
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/1
80257/80257 [==============================] - 2s - loss: 3.1409 - acc: 0.2575 - val_loss: 2.9965 - val_acc: 0.2591


In [91]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_2 (Embedding)          (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________________________

In [92]:
model.optimizer.lr = 1e-2
model.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=6,
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/6
80257/80257 [==============================] - 2s - loss: 2.3486 - acc: 0.3071 - val_loss: 2.7243 - val_acc: 0.2742
Epoch 2/6
80257/80257 [==============================] - 2s - loss: 2.0937 - acc: 0.3285 - val_loss: 2.6624 - val_acc: 0.2804
Epoch 3/6
80257/80257 [==============================] - 2s - loss: 1.9939 - acc: 0.3436 - val_loss: 2.6489 - val_acc: 0.2855
Epoch 4/6
80257/80257 [==============================] - 2s - loss: 1.9513 - acc: 0.3528 - val_loss: 2.6508 - val_acc: 0.2898
Epoch 5/6
80257/80257 [==============================] - 2s - loss: 1.9279 - acc: 0.3609 - val_loss: 2.6606 - val_acc: 0.2880
Epoch 6/6
80257/80257 [==============================] - 2s - loss: 1.9131 - acc: 0.3666 - val_loss: 2.6752 - val_acc: 0.2874


In [93]:
model.optimizer.lr = 1e-3
model.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=6,
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/6
80257/80257 [==============================] - 2s - loss: 1.9011 - acc: 0.3732 - val_loss: 2.6869 - val_acc: 0.2915
Epoch 2/6
80257/80257 [==============================] - 2s - loss: 1.8898 - acc: 0.3801 - val_loss: 2.6999 - val_acc: 0.2902
Epoch 3/6
80257/80257 [==============================] - 2s - loss: 1.8824 - acc: 0.3859 - val_loss: 2.7102 - val_acc: 0.2882
Epoch 4/6
80257/80257 [==============================] - 2s - loss: 1.8721 - acc: 0.3916 - val_loss: 2.7283 - val_acc: 0.2886
Epoch 5/6
80257/80257 [==============================] - 2s - loss: 1.8669 - acc: 0.3966 - val_loss: 2.7358 - val_acc: 0.2912
Epoch 6/6
80257/80257 [==============================] - 2s - loss: 1.8589 - acc: 0.4007 - val_loss: 2.7467 - val_acc: 0.2869


## Adding Bias Term

In [123]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [124]:
ub = Embedding(n_users, 1, input_length=1)(user_in)
mb = Embedding(n_movies, 1, input_length=1)(movie_in)
ub = Flatten()(ub)
mb = Flatten()(mb)

In [125]:
x = merge([u, m], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, mb], mode='sum')
model = Model([user_in, movie_in], x)
model.compile(Adam(1e-3), loss='mse', metrics=['accuracy'])


In [126]:
print(n_users)
print(n_movies)
print(n_users * 51 + n_movies * 51)
model.summary()


671
9066
496587
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
user_in (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
movie_in (InputLayer)            (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_27 (Embedding)         (None, 1, 50)         33550       user_in[0][0]                    
____________________________________________________________________________________________________
embedding_28 (Embedding)         (None, 1, 50)         453300      movie_in[0][0]                   
___________________________________________________________________________

In [127]:
model.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=1,
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/1
80257/80257 [==============================] - 3s - loss: 8.6488 - acc: 0.0486 - val_loss: 3.4593 - val_acc: 0.1593


In [128]:
model.optimizer.lr = 1e-2
model.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=6,
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/6
80257/80257 [==============================] - 3s - loss: 2.5609 - acc: 0.2247 - val_loss: 2.3184 - val_acc: 0.2590
Epoch 2/6
80257/80257 [==============================] - 3s - loss: 2.0007 - acc: 0.2802 - val_loss: 2.1189 - val_acc: 0.2820
Epoch 3/6
80257/80257 [==============================] - 3s - loss: 1.8430 - acc: 0.2965 - val_loss: 2.0241 - val_acc: 0.2879
Epoch 4/6
80257/80257 [==============================] - 3s - loss: 1.7486 - acc: 0.3039 - val_loss: 1.9469 - val_acc: 0.2924
Epoch 5/6
80257/80257 [==============================] - 3s - loss: 1.6696 - acc: 0.3085 - val_loss: 1.8763 - val_acc: 0.2941
Epoch 6/6
80257/80257 [==============================] - 3s - loss: 1.5946 - acc: 0.3143 - val_loss: 1.8081 - val_acc: 0.2965


In [129]:
model.optimizer.lr = 1e-3
model.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=6,
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/6
80257/80257 [==============================] - 3s - loss: 1.5235 - acc: 0.3198 - val_loss: 1.7442 - val_acc: 0.3010
Epoch 2/6
80257/80257 [==============================] - 3s - loss: 1.4539 - acc: 0.3269 - val_loss: 1.6851 - val_acc: 0.3018
Epoch 3/6
80257/80257 [==============================] - 3s - loss: 1.3857 - acc: 0.3336 - val_loss: 1.6239 - val_acc: 0.3039
Epoch 4/6
80257/80257 [==============================] - 3s - loss: 1.3187 - acc: 0.3424 - val_loss: 1.5668 - val_acc: 0.3085
Epoch 5/6
80257/80257 [==============================] - 3s - loss: 1.2528 - acc: 0.3527 - val_loss: 1.5148 - val_acc: 0.3101
Epoch 6/6
80257/80257 [==============================] - 3s - loss: 1.1897 - acc: 0.3617 - val_loss: 1.4655 - val_acc: 0.3139


## Analyze results

In [147]:
g = ratings.groupby('movieIdx')['rating'].count()
topMovies = g.sort_values(ascending=False)[:2000]
topMovies = np.array(topMovies.index)

In [156]:
get_movie_bias = Model(movie_in, mb)
movie_bias = get_movie_bias.predict(topMovies)
movie_ratings = [(b[0], movie_names[movieIds[i]]) for i,b in zip(topMovies,movie_bias)]

In [161]:
movie_bias

array([[ 0.6989],
       [ 0.6952],
       [ 0.7712],
       ..., 
       [ 0.5666],
       [ 0.3025],
       [ 0.3903]], dtype=float32)

In [157]:
sorted(movie_ratings, key=itemgetter(0))[:15]

[(-0.011868457, 'Battlefield Earth (2000)'),
 (0.033481024, 'Little Nicky (2000)'),
 (0.045714505, 'Super Mario Bros. (1993)'),
 (0.056595542, 'Batman & Robin (1997)'),
 (0.078083545, 'Speed 2: Cruise Control (1997)'),
 (0.084160037, 'Police Academy 6: City Under Siege (1989)'),
 (0.086015686, 'Blame It on Rio (1984)'),
 (0.096223675, '2 Fast 2 Furious (Fast and the Furious 2, The) (2003)'),
 (0.11228503, 'House on Haunted Hill (1999)'),
 (0.12177217, 'Wild Wild West (1999)'),
 (0.12956317, 'Lost in Space (1998)'),
 (0.13219632, 'Rambo III (1988)'),
 (0.13306664, 'Two Weeks Notice (2002)'),
 (0.13578936, 'Spice World (1997)'),
 (0.13916759, "Charlie's Angels: Full Throttle (2003)")]

In [159]:
sorted(movie_ratings, key=itemgetter(0), reverse=True)[:15]

[(0.90871221, 'Cyrano de Bergerac (1990)'),
 (0.84258831, 'Gold Rush, The (1925)'),
 (0.83125675, 'Perks of Being a Wallflower, The (2012)'),
 (0.83050346, 'Last King of Scotland, The (2006)'),
 (0.80548501, 'Argo (2012)'),
 (0.79542851, 'Exotica (1994)'),
 (0.79015726, 'My Neighbor Totoro (Tonari no Totoro) (1988)'),
 (0.78081292, 'Mississippi Burning (1988)'),
 (0.78013265, 'Dallas Buyers Club (2013)'),
 (0.77668571, 'Wolf of Wall Street, The (2013)'),
 (0.7763688, 'Tom Jones (1963)'),
 (0.77524644, 'Gone Baby Gone (2007)'),
 (0.77122247, 'Shawshank Redemption, The (1994)'),
 (0.77121168, '12 Years a Slave (2013)'),
 (0.76978338, 'Captain America: The Winter Soldier (2014)')]

In [160]:
get_movie_emb = Model(movie_in, m)
movie_emb = np.squeeze(get_movie_emb.predict([topMovies]))
movie_emb.shape

(2000, 50)

In [162]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
movie_pca = pca.fit(movie_emb.T).components_

In [165]:
fac0 = movie_pca[0]

In [ ]:
movie_comp = [(f, movie_names[movieIds[i]]) for f,i in zip(fac0, topMovies)]

### Not finished

## Neural Net

In [175]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users, n_factors, input_length=1, W_regularizer=l2(1e-4))(user_in)
movie_in = Input(shape=(1,), dtype='int64', name='movie_in')
m = Embedding(n_movies, n_factors, input_length=1, W_regularizer=l2(1e-4))(movie_in)

In [178]:
x = merge([u, m], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn = Model([user_in, movie_in], x)
nn.compile(Adam(0.001), loss='mse', metrics=['accuracy'])

In [174]:
nn.fit([trn.userIdx, trn.movieIdx], trn.rating, batch_size=64, nb_epoch=8, 
          validation_data=([val.userIdx, val.movieIdx], val.rating))

Train on 80257 samples, validate on 19747 samples
Epoch 1/8
80257/80257 [==============================] - 3s - loss: 2.5088 - acc: 0.1891 - val_loss: 0.9308 - val_acc: 0.2839
Epoch 2/8
80257/80257 [==============================] - 3s - loss: 1.5073 - acc: 0.2328 - val_loss: 0.8871 - val_acc: 0.3014
Epoch 3/8
80257/80257 [==============================] - 3s - loss: 1.2413 - acc: 0.2576 - val_loss: 0.8594 - val_acc: 0.3155
Epoch 4/8
80257/80257 [==============================] - 3s - loss: 1.0307 - acc: 0.2869 - val_loss: 0.8644 - val_acc: 0.3169
Epoch 5/8
80257/80257 [==============================] - 3s - loss: 0.9002 - acc: 0.3117 - val_loss: 0.8432 - val_acc: 0.3283
Epoch 6/8
80257/80257 [==============================] - 3s - loss: 0.8365 - acc: 0.3249 - val_loss: 0.8433 - val_acc: 0.3299
Epoch 7/8
80257/80257 [==============================] - 3s - loss: 0.8162 - acc: 0.3363 - val_loss: 0.8466 - val_acc: 0.3330
Epoch 8/8
80257/80257 [==============================] - 3s - loss: 